# import lib

In [1]:
import sys
sys.path.append('/Users/satyammalhotra/miniforge3/envs/tensor25/lib/python3.8/os.py')
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [5]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Data 

In [6]:
mnist_dataset,mnist_info = tfds.load(name='mnist',with_info=True,as_supervised=True)

Metal device set to: Apple M1


2021-12-03 14:58:55.203873: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-03 14:58:55.205048: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
mnist_train,mnist_test = mnist_dataset['train'],mnist_dataset['test']



# total data 70k, training 60, test 10, validation 10% of training 6k

In [8]:
num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples,tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples,tf.int64)


def scale(image,label):
    image = tf.cast(image,tf.float32)
    image /=255.
    return image,label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)


train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE=100

train_data = train_data.batch(BATCH_SIZE) 
validation_data = validation_data.batch(num_validation_samples) 
test_data = test_data.batch(num_test_samples)

validation_inputs,validation_targets = next(iter(validation_data)) 

2021-12-03 14:59:01.166697: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-03 14:59:01.482071: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


# Model

# Outline the model

In [11]:
input_size = 784
output_size = 10
hidden_layer_size = 50


model1 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(output_size,activation='softmax')
])

# now done with 1)data 2)model now go for 3) objective function

# ______________________________________________________________

# Choose the Optimizer and the loss functio

# adam is same as gd but it varies learning rate at each update and for each weight infividually`


In [12]:
model1.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

# Training

In [14]:
NUM_EPOCHS = 5
model1.fit(train_data,epochs=NUM_EPOCHS,validation_data=(validation_inputs,validation_targets),verbose=2)

Epoch 1/5


2021-12-03 17:05:58.737006: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-03 17:06:03.784428: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


540/540 - 6s - loss: 0.4165 - accuracy: 0.8831 - val_loss: 0.2100 - val_accuracy: 0.9398 - 6s/epoch - 11ms/step
Epoch 2/5
540/540 - 3s - loss: 0.1823 - accuracy: 0.9462 - val_loss: 0.1557 - val_accuracy: 0.9540 - 3s/epoch - 6ms/step
Epoch 3/5
540/540 - 3s - loss: 0.1385 - accuracy: 0.9595 - val_loss: 0.1248 - val_accuracy: 0.9635 - 3s/epoch - 6ms/step
Epoch 4/5
540/540 - 4s - loss: 0.1135 - accuracy: 0.9668 - val_loss: 0.1070 - val_accuracy: 0.9713 - 4s/epoch - 7ms/step
Epoch 5/5
540/540 - 4s - loss: 0.0928 - accuracy: 0.9724 - val_loss: 0.0899 - val_accuracy: 0.9732 - 4s/epoch - 7ms/step


# Testing the Model


In [15]:
test_loss,test_accuracy=model1.evaluate(test_data)
print('Test loss:{0:.2f}, test accuracy:{1:.2f}%'.format(test_loss,test_accuracy*100.))

1/1 [==============================] - 1s 606ms/step - loss: 0.1095 - accuracy: 0.9662
Test loss:0.11, test accuracy:96.62%
